In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sim.dy.model_intv import ModelIntv
import sim.dy.keys as I

In [2]:
def get_r_acf(pars_post):
    r_acf = []

    for p in pars_post:
        _, pa = m.find_baseline(p, 2022)
        r_acf.append({'r_acf_mu': pa['r_acf_mu'], 'r_acf_d2d': pa['r_acf_d2d']})

    return {
        'r_acf_mu': np.mean([r['r_acf_mu'] for r in r_acf]), 
        'r_acf_d2d': np.mean([r['r_acf_d2d'] for r in r_acf])
    }

In [3]:
m = ModelIntv()

In [4]:
for folder in ['dy_lo']:
# for folder in ['dy_free', 'dy_lo', 'dy_hi']:
    print(folder)
    out_path = f'out/{folder}'
    pars_post = json.load(open(f'{out_path}/Post.json', 'r'))
    r_acf = get_r_acf(pars_post)

    with open(f'{out_path}/R_ACF_Bg.json', 'w') as f:
        json.dump(r_acf, f)